In [9]:
#hex char to binary
def hex_to_binary(hex_char):
    try:
        hex_int = int(hex_char, 16)
        binary_str = bin(hex_int)[2:]
        binary_str = binary_str.zfill(4)
        return binary_str
    except ValueError:
        return "Invalid hex character"

#binary converstion to s_box
def s_box_substitution(binary_str):
    return s_box.get(binary_str, "Invalid binary string")

s_box = {
    '0000': '1010', '0001': '0000', '0010': '1001', '0011': '1110',
    '0100': '0110', '0101': '0011', '0110': '1111', '0111': '0101',
    '1000': '0001', '1001': '1101', '1010': '1100', '1011': '0111',
    '1100': '1011', '1101': '0100', '1110': '0010', '1111': '1000'
}

#binary string to hex char
def binary_to_hex(binary_str):
    try:
        binary_str = binary_str.zfill(4)
        hex_int = int(binary_str, 2)
        hex_char = hex(hex_int)[2:]
        
        return hex_char
    except ValueError:
        return "Invalid binary string"

#list of binary strings to hex string
def binary_list_to_hex(binary_list):
    return ''.join([binary_to_hex(binary) for binary in binary_list])

#direct converstion of given hex string list to binary strings
def hex_to_binary_list(hex_string):
    hex_string = hex_string.zfill(4)
    
    binary_result = []
    for hex_char in hex_string:
        binary_char = hex_to_binary(hex_char)
        binary_result.append(binary_char)
    
    return binary_result

#bitwise XOR of 2 given hex characters
def hex_xor(hex_char1, hex_char2):
    try:
        int1 = int(hex_char1, 16)
        int2 = int(hex_char2, 16)
        result = int1 ^ int2
        hex_result = hex(result)[2:]
        
        return hex_result.zfill(1)
    except ValueError:
        return "Invalid hex character"

# Example usage:
hex_char1 = 'A'
hex_char2 = '5'
xor_result = hex_xor(hex_char1, hex_char2)
print(f"Hex XOR: {hex_char1} XOR {hex_char2} = {xor_result}")



Hex XOR: A XOR 5 = f


# Input to Sub Nibble

In [10]:
#direct converstion of given hex string to s_box converted hex string
def transform_hex_string(hex_string):
    hex_string = hex_string.zfill(4)
    
    binary_result = []
    for hex_char in hex_string:
        binary_char = hex_to_binary(hex_char)
        s_box_result = s_box_substitution(binary_char)
        binary_result.append(s_box_result)
    
    return binary_result

hex_string = '903b'
hex_result = binary_list_to_hex(transform_hex_string(hex_string))
print(f"Sub Nibble: {hex_result}")

Sub Nibble: dae7


# Generating Keys

In [11]:
#conversion of list of binary strings to list of hex chararacters
def binary_list_to_hex_list(binary_list):
    try:
        hex_list = [hex(int(binary, 2))[2:].zfill(1) for binary in binary_list]
        return hex_list
    except ValueError:
        return "Invalid binary string in the list"

#generating key1 and key2
def generate_round_keys(initial_key):
    key_in_binary = hex_to_binary_list(initial_key)
    hex_char_list = binary_list_to_hex_list(key_in_binary)
    
    Rcon1 = '1110'
    Rcon2 = '1010'
    Rcon1 = binary_to_hex(Rcon1)
    Rcon2 = binary_to_hex(Rcon2)
    
    sub_nibble_w3 = binary_to_hex(s_box_substitution(key_in_binary[3]))
    
    w4 = hex_xor(hex_char_list[0], sub_nibble_w3)
    w4 = hex_xor(w4, Rcon1)
    w5 = hex_xor(hex_char_list[1], w4)
    w6 = hex_xor(hex_char_list[2], w5)
    w7 = hex_xor(hex_char_list[3], w6)
    
    k1 = ''.join([w4, w5, w6, w7])
    
    sub_nibble_w7 = binary_to_hex(s_box_substitution(hex_to_binary(w7)))
    
    w8 = hex_xor(w4, sub_nibble_w7)
    w8 = hex_xor(w8, Rcon2)
    w9 = hex_xor(w5, w8)
    w10 = hex_xor(w6, w9)
    w11 = hex_xor(w7, w10)
    
    k2 = ''.join([w8, w9, w10, w11])
    
    return [k1, k2]
    
print(generate_round_keys('2cc'))

['57b7', 'ad61']


# Shift Row 

In [12]:
def shift_row(hex_string):
    key_in_binary = hex_to_binary_list(hex_string)
    hex_char_list = binary_list_to_hex_list(key_in_binary)
    hex_char_list[0], hex_char_list[2] = hex_char_list[2], hex_char_list[0]
    
    return ''.join(hex_char_list)

shift_row('903b')

'309b'

# Mix Columns

In [13]:
def multiply_in_gf(a, b):
    result = '0000'  
    a = int(a, 2) 
    b = int(b, 2)  
    for _ in range(4):
        if b & 1:
            result = bin(int(result, 2) ^ a)[2:].zfill(4) 
        a <<= 1
        if a & 0b10000:
            a ^= 0b10011
        b >>= 1
    return result

#int list to binary list
def int_list_to_binary_matrix(int_list):
    binary_matrix = []
    for num in int_list:
        binary_str = bin(num)[2:].zfill(4)
        binary_matrix.append(binary_str)
    return binary_matrix

def mix_columns(hex_string):
    key_in_binary = hex_to_binary_list(hex_string)
    hex_char_list = binary_list_to_hex_list(key_in_binary)
    
    matrix = int_list_to_binary_matrix([1, 4, 4, 1])
    
    d0 = hex_xor(multiply_in_gf(matrix[0], key_in_binary[0]), multiply_in_gf(matrix[2], key_in_binary[1]))
    d1 = hex_xor(multiply_in_gf(matrix[1], key_in_binary[0]), multiply_in_gf(matrix[3], key_in_binary[1]))
    
    d2 = hex_xor(multiply_in_gf(matrix[0], key_in_binary[2]), multiply_in_gf(matrix[2], key_in_binary[3]))
    d3 = hex_xor(multiply_in_gf(matrix[1], key_in_binary[2]), multiply_in_gf(matrix[3], key_in_binary[3]))
    
    return ''.join(binary_list_to_hex_list([d0, d1, d2, d3]))

mix_columns('903b') #9297


'9297'

# Round 1

In [14]:
#index wise XOR of hex characters
def xor_lists(list1, list2):
    result = []
    for elem1, elem2 in zip(list1, list2):
        result.append(hex_xor(elem1, elem2))
    return result

hex_string = '32ef'
def round_1(hex_string):
    hex_result = binary_list_to_hex(transform_hex_string(hex_string))
    keys = generate_round_keys('2cc')
    print(f"Input Hex: {''.join(hex_string)}")

    hex_in_binary = hex_to_binary_list(hex_result)
    hex_char_list = binary_list_to_hex_list(hex_in_binary) 
    print(f"Sub Nibble: {''.join(hex_char_list)}")
    
    key_in_binary = hex_to_binary_list(keys[0])
    key_char_list = binary_list_to_hex_list(key_in_binary)
    print(f"Input key list: {''.join(key_char_list)}")
    
    sub_nibble_plus_key = xor_lists(hex_char_list, key_char_list)
    print(f"Sub nibble plus key: {''.join(sub_nibble_plus_key)}")
    
    mixed_columns = mix_columns(''.join(sub_nibble_plus_key))
    print(f"Mixed Colums: {mixed_columns}")
    
    shifted_row = shift_row(mixed_columns)
    print(f"Shifted row: {shifted_row}")
    
    return shifted_row

round_1_result = round_1(hex_string)

Input Hex: 32ef
Sub Nibble: e928
Input key list: 57b7
Sub nibble plus key: be9f
Mixed Colums: 640d
Shifted row: 046d


# Round 2

In [15]:
def round_2(round_1_result):
    hex_result = binary_list_to_hex(transform_hex_string(round_1_result))
    keys = generate_round_keys('2cc')
    print(f"Input Hex: {''.join(round_1_result)}")
    
    hex_in_binary = hex_to_binary_list(hex_result)
    hex_char_list = binary_list_to_hex_list(hex_in_binary) 
    print(f"Sub Nibble: {''.join(hex_char_list)}")
    
    key_in_binary = hex_to_binary_list(keys[1])
    key_char_list = binary_list_to_hex_list(key_in_binary)
    print(f"Input key list: {''.join(key_char_list)}")
    
    sub_nibble_plus_key = xor_lists(hex_char_list, key_char_list)
    print(f"Sub nibble plus key: {''.join(sub_nibble_plus_key)}")
    
    shifted_row = shift_row(''.join(sub_nibble_plus_key))
    print(f"Shifted row: {shifted_row}")
    
    return shifted_row
   
round_2_result = round_2(round_1_result)

Input Hex: 046d
Sub Nibble: a6f4
Input key list: ad61
Sub nibble plus key: 0b95
Shifted row: 9b05


In [26]:
def sub_nibbles(round_1_result):
    return binary_list_to_hex(transform_hex_string(round_1_result))

text_block = input("Enter a text block: ") #903b
print(sub_nibbles(text_block))
print(shift_row(text_block))
print(mix_columns(text_block))

key = input("Enter a key: ") #2cc
generate_round_keys(key) 

Enter a text block: 43ba
6e7c
b34a
8050
Enter a key: 1122


['6757', '9ebc']

# Decryption

# Inverse of Sub Nibble

In [18]:
def inverse_s_box_substitution(binary_str):
    reversed_s_box = {
        '1010': '0000', '0000': '0001', '1001': '0010', '1110': '0011',
        '0110': '0100', '0011': '0101', '1111': '0110', '0101': '0111',
        '0001': '1000', '1101': '1001', '1100': '1010', '0111': '1011',
        '1011': '1100', '0100': '1101', '0010': '1110', '1000': '1111'
    }
    return reversed_s_box.get(binary_str, "Invalid binary string")

#direct converstion of given hex string to s_box converted hex string
def inverse_transform_hex_string(hex_string):
    hex_string = hex_string.zfill(4)
    
    binary_result = []
    for hex_char in hex_string:
        binary_char = hex_to_binary(hex_char)
        s_box_result = inverse_s_box_substitution(binary_char)
        binary_result.append(s_box_result)
    
    return binary_result

hex_string = 'dae7'
hex_result = binary_list_to_hex(inverse_transform_hex_string(hex_string))
print(f"Inverse Sub Nibble: {hex_result}")

Inverse Sub Nibble: 903b


# Inverse mix columns

In [19]:
def inverse_mix_columns(hex_string):
    key_in_binary = hex_to_binary_list(hex_string)
    hex_char_list = binary_list_to_hex_list(key_in_binary)
    
    matrix = int_list_to_binary_matrix([9, 2, 2, 9])
    
    d0 = hex_xor(multiply_in_gf(matrix[0], key_in_binary[0]), multiply_in_gf(matrix[2], key_in_binary[1]))
    d1 = hex_xor(multiply_in_gf(matrix[1], key_in_binary[0]), multiply_in_gf(matrix[3], key_in_binary[1]))
    
    d2 = hex_xor(multiply_in_gf(matrix[0], key_in_binary[2]), multiply_in_gf(matrix[2], key_in_binary[3]))
    d3 = hex_xor(multiply_in_gf(matrix[1], key_in_binary[2]), multiply_in_gf(matrix[3], key_in_binary[3]))
    
    return ''.join(binary_list_to_hex_list([d0, d1, d2, d3]))

inverse_mix_columns('9297') #903b

'903b'

In [27]:
def decryption(round_2_result, keyInput):
    hex_result = binary_list_to_hex(hex_to_binary_list(round_2_result))                                            
    keys = generate_round_keys(keyInput)
    
    #Round 2 Inverse
    hex_in_binary = hex_to_binary_list(round_2_result)
    hex_char_list = binary_list_to_hex_list(hex_in_binary) 
    print(f"Input Hex list: {''.join(hex_char_list)}")
    
    key_in_binary = hex_to_binary_list(keys[1])
    key_char_list = binary_list_to_hex_list(key_in_binary)
    print(f"Input key list (round2) : {''.join(key_char_list)}")
    
    inverse_shift_row = shift_row(round_2_result)
    print(f"Inverse of Shift Row (round2): {inverse_shift_row}")
    
    inverse_shift_row_plus_key = xor_lists(inverse_shift_row, key_char_list)
    print(f"Inverse of input key (round2): {''.join(inverse_shift_row_plus_key)}")
    
    hex_result = binary_list_to_hex(inverse_transform_hex_string(''.join(inverse_shift_row_plus_key)))
    print(f"Inverse of Sub Nibble (round2): {hex_result}")
    
    print("\nRound 1\n")
    
    #Round 1 Inverse
    inverse_shift_row = shift_row(''.join(hex_result))
    print(f"Inverse of Shift Row (round1): {inverse_shift_row}")
    
    inverse_mixed_columns = inverse_mix_columns(''.join(inverse_shift_row))
    print(f"Inverse Mixed Colums: {inverse_mixed_columns}")
    
    key_in_binary = hex_to_binary_list(keys[0])
    key_char_list = binary_list_to_hex_list(key_in_binary)
    print(f"Input key list (round1): {''.join(key_char_list)}")
    
    inverse_mixed_columns_plus_key = xor_lists(binary_list_to_hex(hex_to_binary_list(inverse_mixed_columns)), key_char_list)
    print(f"Inverse of input key (round1): {''.join(inverse_mixed_columns_plus_key)}")
    
    hex_result = binary_list_to_hex(inverse_transform_hex_string(''.join(inverse_mixed_columns_plus_key)))
    print(f"Inverse of Sub Nibble (round1): {hex_result}")
    
    return hex_result

text_block = input("Enter a text block: ") #903b
key = input("Enter a key: ") #2cc

decryption(text_block, key)

Enter a text block: 9213
Enter a key: a321
Input Hex list: 9213
Input key list (round2) : 8fae
Inverse of Shift Row (round2): 1293
Inverse of input key (round2): 9d3d
Inverse of Sub Nibble (round2): 2959

Round 1

Inverse of Shift Row (round1): 5929
Inverse Mixed Colums: a709
Input key list (round1): 4754
Inverse of input key (round1): e05d
Inverse of Sub Nibble (round1): 3179


'3179'

# D3

In [28]:
try:
    with open("secret.txt", "r") as file:
        data = file.read()
        
    values = data.split()
    processed_values = []

    for value in values:
        if len(value) == 2:
            value += "00"
        processed_values.append(value)

except FileNotFoundError:
    print("The file 'secret.txt' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

key = input("Enter the key: ") #149c

decrypted_values = []

for value in processed_values:
        decrypted_value = decryption(value, key)
        decrypted_values.append(decrypted_value)

Enter the key: 149c
Input Hex list: 7995
Input key list (round2) : dd41
Inverse of Shift Row (round2): 9975
Inverse of input key (round2): 4434
Inverse of Sub Nibble (round2): dd5d

Round 1

Inverse of Shift Row (round1): 5ddd
Inverse Mixed Colums: 2566
Input key list (round1): 4095
Inverse of input key (round1): 65f3
Inverse of Sub Nibble (round1): 4765
Input Hex list: ac7e
Input key list (round2) : dd41
Inverse of Shift Row (round2): 7cae
Inverse of input key (round2): a1ef
Inverse of Sub Nibble (round2): 0836

Round 1

Inverse of Shift Row (round1): 3806
Inverse Mixed Colums: b2c3
Input key list (round1): 4095
Inverse of input key (round1): f256
Inverse of Sub Nibble (round1): 6e74
Input Hex list: 4d95
Input key list (round2) : dd41
Inverse of Shift Row (round2): 9d45
Inverse of input key (round2): 4004
Inverse of Sub Nibble (round2): d11d

Round 1

Inverse of Shift Row (round1): 11dd
Inverse Mixed Colums: bb66
Input key list (round1): 4095
Inverse of input key (round1): fbf3
Invers

In [29]:
def hex_to_ascii(hex_string):
    try:
        ascii_value = int(hex_string, 16)
        if 0 <= ascii_value <= 127:
            ascii_char = chr(ascii_value)
            return ascii_char
        else:
            return "Invalid ASCII value"
    except ValueError:
        return "Invalid input hex string"

decrypted_values_expanded = []
for each in decrypted_values:
    decrypted_values_expanded.append(each[0] + each[1])
    decrypted_values_expanded.append(each[2] + each[3])

plain_text_file_content = []
for index, each in enumerate(decrypted_values_expanded):
    if index != len(decrypted_values_expanded) - 1:
        plain_text_file_content.append(hex_to_ascii(each))
    elif each != '00':
        plain_text_file_content.append(hex_to_ascii(each))

output = ''.join(plain_text_file_content)
output

"Gentlemen, you can't fight in here. This is the war room."

# Saving to plain.txt

In [30]:
file_name = 'plain.txt'
with open(file_name, 'w') as file:
    file.write(output)